In [1]:
%pwd

'd:\\Projects\\Gen AI\\Medical_chatbot_1\\research'

In [2]:
import os 
os.chdir("../")
%pwd

'd:\\Projects\\Gen AI\\Medical_chatbot_1'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf_files("data")


In [6]:
extracted_data[20:23]

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\Medical_book.pdf', 'total_pages': 637, 'page': 20, 'page_label': '21'}, page_content='the late second or third trimester. Although D&X is high-\nly controversial, some physicians argue that it has advan-\ntages that make it a preferable procedure in some circum-\nstances. One perceived advantage is that the fetus is\nremoved largely intact, allowing for better evaluation and\nautopsy of the fetus in cases of known fetal anomalies.\nIntact removal of the fetus may also confer a lower risk\nof puncturing the uterus or damaging the cervix. Another\nperceived advantage is that D&X ends the pregnancy\nwithout requiring the woman to go through labor, which\nmay be less emotionally traumatic than other methods of\nlate-term abortion. In addition, D&X may offer a lower\ncost and shorter procedure time.\nPrecautions

In [7]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter. split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()
embedding

C:\Users\anime\AppData\Local\Temp\ipykernel_26976\3442840130.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
d:\Projects\Gen AI\Medical_chatbot_1\genai_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [10]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [11]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY
pc = Pinecone(api_key=pinecone_api_key)
pc

In [12]:
from pinecone import ServerlessSpec

index_name = "medical-chatbot-1"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)


In [13]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embedding,
    index_name=index_name
)

In [14]:
print(dir(docsearch))
docsearch_index = docsearch._index  # this is the actual Pinecone index
print(docsearch_index.describe_index_stats())


['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_asimilarity_search_with_relevance_scores', '_async_index', '_cosine_relevance_score_fn', '_embedding', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_index', '_index_host', '_max_inner_product_relevance_score_fn', '_namespace', '_pinecone_api_key', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', '_text_key', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_texts', 'adelete', 'afrom_documents', 'afrom_texts', 'aget_by_ids', 'amax_marginal_relevance_search', 'amax_marginal_relevance_search_by_vector', 'as_retriever', 'asear

In [15]:
retriever = docsearch.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.75})

In [16]:
retrieved_docs = retriever.invoke("information about vomiting")
retrieved_docs

[Document(id='f525bbd9-d8b5-401a-9b6b-64e0d98de5a1', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 226.0, 'page_label': '227', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='tives to clear the stomach and intestines after a\nbinge.\nRussell’s sign—Scraped or raw areas on the patient’s\nknuckles, caused by self-induced vomiting.\nSuperior mesenteric artery syndrome —A condi-\ntion in which a person vomits after meals due to\nblockage of the blood supply to the intestine.\nBaron, Robert B. “Nutrition.” In Current Medical Diagnosis\nand Treatment, 1998.37th ed. Ed. Stephen McPhee, et al.\nStamford: Appleton & Lange, 1997.'),
 Document(id='f1aa4199-a70a-4932-941e-d68764c162a6', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 226.0, 'page_label': '227', 'producer': 'PDFlib+PDI

In [17]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

chatModel = ChatOpenAI(
    model="deepseek/deepseek-chat",    # ✅ safe working model
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    timeout=30,                        # ⏱ fail fast instead of hanging forever
    max_retries=1,                     # no endless retries
    default_headers={
        "HTTP-Referer": "http://localhost:8501",
        "X-Title": "Medical Chatbot 1"
    },
)

print("Sending request...")
resp = chatModel.invoke("Quick test: say hello if you're working! And tell me are you coming from OpenRouter, if no then from where?")
print("Response:", resp.content)


Sending request...
Response: Hello! I'm working fine. 😊  

I'm not coming from OpenRouter—I'm hosted directly by DeepSeek. Let me know how I can help! 🚀


In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [19]:
system_prompt = (
    "You are an First-Aid Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Keep the answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [20]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [21]:
response = rag_chain.invoke({"input": "information about sore tonsils"})
print(response["answer"])

Sore tonsils, often due to acute tonsillitis, can be caused by viral or bacterial infections (e.g., cold viruses, strep throat, or mononucleosis). Symptoms include pain, redness, and swelling. Warm saline gargles may help alleviate discomfort. Prompt evaluation and treatment of sore throats can prevent complications. If symptoms persist or cause breathing difficulties, medical intervention may be needed.
